## Hypothesis

Annual circulation can be predicted by library and geographical characteristics.

## Data

The data is from the 2014 Public Library Survey, the most recent available survey from the Institute of Museum and Library Services.  Three CSV files are publicly available, broken down by state, administrative entity, and library outlet. I will be using the administrative entity data set since it’s the most comprehensive.  There are 9,305 entries and 159 variables.
https://www.imls.gov/research-evaluation/data-collection/public-libraries-survey/explore-pls-data/pls-data

Additionally, I added annual median household income data, broken down by state, from the 2014 American Community Survey 5-year estimates.
https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_15_5YR_S1903&prodType=table

### Data Dictionary

The data dictionary for the Public Library Survey can be found here, starting at page 24 of the pdf.
https://www.imls.gov/sites/default/files/fy2014_pls_data_file_documentation.pdf

A data dictionary for the variables I will be using is below.  All variables, except STINCOME, comes from the Public Library Survey.

Variable Name | Description | Type of Variable
:---: | :---| :---
TOTCIR | Total annual circulation | continuous
POPU_LSA | Population of the legal service area | Continuous
STINCOME | Household median income (dollars) | Continuous
BKVOL | Print materials |  Continuous
EBOOK | Electronic books | Continuous
STGVT | Operating revenue from state government | Continuous
FEDGVT | Operating revenue from federal government | Continuos
LOCGVT | Operating revenue from local government | Continuous
TOTSTAFF | Total paid full time employees | Continuous
GEOCODE | Geographic Code| Categorical
OBEREG | Region | Categorical
LOCALE | Geographic location | Categorical

GEOCODE

Code | Category
:---: | :---
CI1 | Municipal Government (city, town, or village) (exactly)
CI2 | Municipal Government (city, town, or village) (most nearly)
CO1 | County/Parish (exactly)
CO2 | County/Parish (most nearly)
MA1 | Metropolitan Area (exactly)
MA2 | Metropolitan Area (most nearly)
MC1 | Multi-County (exactly)
MC2 | Multi-County (most nearly)
SD1 | School Disctrict (exactly)
SD2 | School District (most nearly)
OTH | Other

OBEREG

Code | Category
:---: | :---
01 | New England (CT, ME, MA, NH, RI, VT)
02 | Mid East (DE, DC, MD, NJ, NY, PA)
03 | Great Lakes (IL, IN, MI, OH, WI)
04 | Plains (IA, KS, MN, MO, NE, ND, SD)
05 | Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC, TN, VA, WV)
06 | Southwest (AZ, NM, OK, TX)
07 | Rocky Mountains (CO, ID, MT, UT, WY)
08 | Far West (AK, CA, HI, NV, OR, WA)
09 | Outlying Areas (AS, GU, MP, PR, VI)

LOCALE

Code | Category
:---: | :---
11 | City, Large: Territory inside an urbanized area and inside a principal city with population of 250,000 or more.
12 | City, Mid-Size: Territory inside an urbanized area and inside a principal city with a population less than 250,000 and greater than or equal to 100,000.
13 | City, Small: Territory inside an urbanized area and inside a principal city with a population less than 100,000.
21 | Suburb, Large: Territory outside a principal city and inside an urbanized area with population of 250,000 or more.
22 | Suburb, Mid-size: Territory outside a principal city and inside an urbanized area with population of less than 250,000 and greater than or equal to 100,000.
23 | Suburb, Small: Territory outside a principal city and inside an urbanized area with population less than 100,000.
31 | Town, Fringe: Territory inside an urban cluster that is less than or equal to 10 miles from an urbanized area.
32 | Town, Distant: Territory inside an urban cluster that is more than 10 miles and less than or equal to 35 miles from an urbanized area.
33 | Town, Remote: Territory inside an urban cluster that is more than 35 miles from an urbanized area.
41 | Rural, Fringe: Census-defined rural territory that is less than or equal to 5 miles from an urbanized area, as well as rural territory that is less than or equal to 2.5 miles from an urban cluster.
42 | Rural, Distant: Census-defined rural territory that is more than 5 miles but less than or equal to 25 miles from an urbanized area, as well as rural territory that is more than 2.5 miles but less than or equal to 10 miles from an urban cluster.
43 | Rural, Remote: Census-defined rural territory that is more than 25 miles from an urbanized area and is also more than 10 miles from an urban cluster.

I found that most of the continuous variables in the dataset were highly correlated, particularly with POPU_LSA. Therefore, I set up models with two sets of features.  The first set predicts TOTCIR using POPU_LSA, STINCOME, STGVT, GEOCODE, OBEREG, and LOCALE. 

For the second set, I divided TOTCIR by POPU_LSA to find the circulation per capita, labeled CIR/POP.  In this model, the features are STINCOME, BKVOL, EBOOK, STGVT, FEDGVT, LOCGVT, TOTSTAFF, GEOCODE, OBEREG, and LOCALE.

I added the STINCOME data by state using Excel to save time and deleted four rows with missing data since it was unllikely these locations would be properly represented by imputing and there were so few missing points.

For everything else, I replaced the missing continuous data with the median and the missing categorical with the mode.  Finally, for all continuous variables other than STINCOME, I added 1 to all data points, so there would be no values of 0, and took the log.  The data in the original form was very skewed.

Finally, I split my data into train and test sets with 80% of the data in the train set and 20% in the test set.

## Analysis

After using a statsmodels linear regression to get a better understanding of both sets of variables, I modeled the data using a linear regression in scikit learn and a random forest.

The code for the set modeling TOTCIR is in Appendix 1 and the code for CIR/POP is in Appendix 2.

### TOTCIR

#### Linear Regression

I used a ridge linear regression on my train data set and, through gridsearch, found that an alpha of 1.0 with an intercept was the best estimator, with a mean squared error of 0.143.  I plotted the predicted values against the actual data as well as the KDE for both.

#### Random Forest

Next, I used a random forest to predict circulation with gridsearch. The best model had a max depth of 27 and min sample split of 30, with a mean squared error of 0.139.  I plotted the predicted values against the actual data and KDE graphs for this model as well, found in Appendix 1.  I also found that the population of the service area was the most important feature.  All other features had very low importance.

### CIR/POP

#### Linear Regression

Using the same process as above, I found that an alpha of 0.1 with an intercept was the best estimator, with a mean squared error of 0.066.  I plotted the predicted values against the actual data as well as the KDE for both.

#### Random Forest

I used gridsearch to optimize a random forest model for this set of variables as well. The best model had a max depth of 22 and min sample split of 20, with a mean squared error of 0.061.  In  this set, EBOOK was the most important feature, followed by STINCOME and LOCGVT.

## Results

In both cases, the random forest model had a slightly lower mean squared error than the linear regression. 

For the first set of models, predicting TOTCIR, plotting the actual y_test values versus the predicted values are more linear than the other models, but I think that is because TOTCIR and POPU_LSA are so correlated.

## Next Steps

The next step would be to include more demographic data, such as area median income by census tract or the age distribution, to see how that affects circulation.  It would be very interesting to see how that affects the circulation.